In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor
from hyperopt import *
from sklearn.metrics import mean_absolute_error
import pyspark 
import mlflow

In [2]:
AFS_SUPERM_17 = pd.read_csv("Distance\\AFS_SUPERM_17_distance.csv", delimiter=',') 
AFS_TREINS_17  = pd.read_csv("Distance\\AFS_TREINS_17_distance.csv", delimiter=',') 
AFS_ONDBAS_17  = pd.read_csv("Distance\\AFS_ONDBAS_17_distance.csv", delimiter=',') 
AFS_ONDVRT_17  = pd.read_csv("Distance\\AFS_ONDVRT_17_distance.csv", delimiter=',') 
AFS_HAPRAK_17  = pd.read_csv("Distance\\AFS_HAPRAK_17_distance.csv", delimiter=',') 
AFS_ZIEK_E_17  = pd.read_csv("Distance\\AFS_ZIEK_E_17_distance.csv", delimiter=',') 
AFS_HORECA_17  = pd.read_csv("Distance\\AFS_HORECA_17_distance.csv", delimiter=',')
woon_17 = pd.read_csv("Distance\\woon_17_distance.csv", delimiter=',') 
winkel_17  = pd.read_csv("Distance\\winkel_17_distance.csv", delimiter=',') 
gezondheid_17  = pd.read_csv("Distance\\gezondheid_17_distance.csv", delimiter=',') 
kantoor_17  = pd.read_csv("Distance\\kantoor_17_distance.csv", delimiter=',') 
bijeenkomst_17  = pd.read_csv("Distance\\bijeenkomst_17_distance.csv", delimiter=',') 
onderwijs_17  = pd.read_csv("Distance\\onderwijs_17_distance.csv", delimiter=',') 
sport_17  = pd.read_csv("Distance\\sport_17_distance.csv", delimiter=',') 
industrie_17  = pd.read_csv("Distance\\industrie_17_distance.csv", delimiter=',') 
logies_17  = pd.read_csv("Distance\\logies_17_distance.csv", delimiter=',')
mean_construction_year_17  = pd.read_csv("Distance\\mean_construction_year_17_distance.csv", delimiter=',') 

woon_18 = pd.read_csv("Distance\\woon_18_distance.csv", delimiter=',') 
winkel_18  = pd.read_csv("Distance\\winkel_18_distance.csv", delimiter=',') 
gezondheid_18  = pd.read_csv("Distance\\gezondheid_18_distance.csv", delimiter=',') 
kantoor_18  = pd.read_csv("Distance\\kantoor_18_distance.csv", delimiter=',') 
bijeenkomst_18  = pd.read_csv("Distance\\bijeenkomst_18_distance.csv", delimiter=',') 
onderwijs_18  = pd.read_csv("Distance\\onderwijs_18_distance.csv", delimiter=',') 
sport_18  = pd.read_csv("Distance\\sport_18_distance.csv", delimiter=',') 
industrie_18  = pd.read_csv("Distance\\industrie_18_distance.csv", delimiter=',') 
logies_18  = pd.read_csv("Distance\\logies_18_distance.csv", delimiter=',')
mean_construction_year_18  = pd.read_csv("Distance\\mean_construction_year_18_distance.csv", delimiter=',') 

woon_19 = pd.read_csv("Distance\\woon_19_distance.csv", delimiter=',') 
winkel_19  = pd.read_csv("Distance\\winkel_19_distance.csv", delimiter=',') 
gezondheid_19  = pd.read_csv("Distance\\gezondheid_19_distance.csv", delimiter=',') 
kantoor_19  = pd.read_csv("Distance\\kantoor_19_distance.csv", delimiter=',') 
bijeenkomst_19  = pd.read_csv("Distance\\bijeenkomst_19_distance.csv", delimiter=',') 
onderwijs_19  = pd.read_csv("Distance\\onderwijs_19_distance.csv", delimiter=',') 
sport_19  = pd.read_csv("Distance\\sport_19_distance.csv", delimiter=',') 
industrie_19  = pd.read_csv("Distance\\industrie_19_distance.csv", delimiter=',') 
logies_19  = pd.read_csv("Distance\\logies_19_distance.csv", delimiter=',') 
mean_construction_year_19  = pd.read_csv("Distance\\mean_construction_year_19_distance.csv", delimiter=',') 

WONING_17  = pd.read_csv("Distance\\WONING_17_distance.csv", delimiter=',') 
WONING_18  = pd.read_csv("Distance\\WONING_18_distance.csv", delimiter=',') 
WONING_19  = pd.read_csv("Distance\\WONING_19_distance.csv", delimiter=',') 

variablelists_17 = [AFS_SUPERM_17, AFS_TREINS_17, AFS_ONDBAS_17, AFS_ONDVRT_17, AFS_HAPRAK_17, AFS_ZIEK_E_17, AFS_HORECA_17, 
                    woon_17, winkel_17,gezondheid_17,kantoor_17,bijeenkomst_17, onderwijs_17, sport_17,industrie_17, logies_17,
                     mean_construction_year_17, WONING_17]
variablelists_18 = [woon_18, winkel_18,gezondheid_18,kantoor_18,bijeenkomst_18, onderwijs_18, sport_18,industrie_18, logies_18,
                     mean_construction_year_18, WONING_18]
variablelists_19 = [woon_19, winkel_19,gezondheid_19,kantoor_19,bijeenkomst_19, onderwijs_19, sport_19,industrie_19, logies_19,
                     mean_construction_year_19, WONING_19]

variablenam_17 = ["AFS_SUPERM_17","AFS_TREINS_17","AFS_ONDBAS_17","AFS_ONDVRT_17","AFS_HAPRAK_17","AFS_ZIEK_E_17","AFS_HORECA_17",
                   "woon_17","winkel_17","gezondheid_17","kantoor_17","bijeenkomst_17","onderwijs_17","sport_17","industrie_17","logies_17",
                   "mean_construction_year_17", "WONING_17"]
variablenam_18 = ["woon_18","winkel_18","gezondheid_18","kantoor_18","bijeenkomst_18","onderwijs_18","sport_18","industrie_18","logies_18",
                   "mean_construction_year_18", "WONING_18"]
variablenam_19 = ["woon_19","winkel_19","gezondheid_19","kantoor_19","bijeenkomst_19","onderwijs_19","sport_19","industrie_19","logies_19",
                   "mean_construction_year_19", "WONING_19"]

In [3]:
def train_val(dataset):
    if len(dataset)> 300000:
        train = dataset.sample(n=225000)
    else:
        train = dataset.sample(frac=0.75)
    test = dataset[~(dataset.c28992r100.isin(train["c28992r100"]))]
    val = test.sample(frac=0.9) 
    val2 = test[~(test.c28992r100.isin(val["c28992r100"]))]
    return train, val, val2

def X_Y(train, val, val2):
    X_train = train.drop(labels=[label,"c28992r100","row","column", "row_1","column_1", "row_2","column_2", "row_3",
                                 "column_3", "row_4","column_4","index"], axis=1)
    X_train = X_train.to_numpy()
    y_train = train[[label]].to_numpy()
    
    X_val = val.drop(labels=[label,"c28992r100","row","column", "row_1","column_1", "row_2","column_2", "row_3",
                                 "column_3", "row_4","column_4","index"], axis=1)
    X_val = X_val.to_numpy()
    y_val = val[[label]].to_numpy()
    
    X_val2 = val2.drop(labels=[label,"c28992r100","row","column", "row_1","column_1", "row_2","column_2", "row_3",
                                 "column_3", "row_4","column_4","index"], axis=1)
    X_val2 = X_val2.to_numpy()
    y_val2 = val2[[label]].to_numpy()
    return X_train, y_train, X_val ,y_val, X_val2 ,y_val2


variablenam = []
for i in variablenam_17:
    variablenam.append(i)
for i in variablenam_18:
    variablenam.append(i)
for i in variablenam_19:
    variablenam.append(i)
    
variablelists = []
for i in variablelists_17:
    variablelists.append(i)
for i in variablelists_18:
    variablelists.append(i)
for i in variablelists_19:
    variablelists.append(i)
    
    
classifiers = []
classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=7, n_estimators=75,
                      n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=4, min_samples_split=8,
                      n_estimators=45, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=4, n_estimators=90,
                      n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=4, n_estimators=70,
                      n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=5, min_samples_split=5,
                      n_estimators=60, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=7, n_estimators=75,
                      n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=10, min_samples_split=23,
                      n_estimators=95, n_jobs=-1)
classifiers.append(classifier)  

classifier = RandomForestRegressor(max_depth=12, min_samples_leaf=5, min_samples_split=23,
                      n_estimators=90, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=12, min_samples_leaf=9, min_samples_split=20,
                      n_estimators=85, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=10, min_samples_leaf=8, min_samples_split=17,
                      n_estimators=30, n_jobs=-1)
classifiers.append(classifier)
    
classifier = RandomForestRegressor(max_depth=8, min_samples_leaf=6, min_samples_split=20,
                      n_estimators=80, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=14, min_samples_leaf=7, min_samples_split=8,
                      n_estimators=35, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=4, min_samples_split=14, n_estimators=10,
                      n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=12, min_samples_leaf=5, min_samples_split=11,
                      n_estimators=40, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=10, min_samples_split=11,
                      n_estimators=85, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=14, min_samples_leaf=9, min_samples_split=14,
                      n_estimators=30, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=14, min_samples_leaf=6, min_samples_split=17,
                      n_estimators=90, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=9, min_samples_split=17,
                      n_estimators=95, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=12, min_samples_leaf=9, min_samples_split=14,
                      n_estimators=50, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=10, min_samples_leaf=6, min_samples_split=8,
                      n_estimators=85, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=12, min_samples_leaf=8, min_samples_split=26,
                      n_estimators=10, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=10, min_samples_split=11,
                      n_estimators=60, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=10, min_samples_leaf=6, min_samples_split=29,
                      n_estimators=80, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=4, min_samples_leaf=8, min_samples_split=23,
                      n_estimators=85, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=12, min_samples_leaf=10, n_estimators=15,
                      n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=10, min_samples_leaf=10, min_samples_split=14,
                      n_estimators=90, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=14, min_samples_leaf=9, min_samples_split=20,
                      n_estimators=20, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=9, min_samples_split=20,
                      n_estimators=90, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=9, min_samples_split=23,
                      n_estimators=70, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=12, min_samples_leaf=9, min_samples_split=26,
                      n_estimators=45, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=12, min_samples_leaf=9, min_samples_split=11,
                      n_estimators=25, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=10, min_samples_leaf=5, min_samples_split=14,
                      n_estimators=10, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=10, min_samples_leaf=7, min_samples_split=26,
                      n_estimators=25, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=12, min_samples_leaf=10, min_samples_split=5,
                      n_estimators=80, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=10, min_samples_leaf=8, min_samples_split=5,
                      n_estimators=15, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=14, min_samples_leaf=2, min_samples_split=5,
                      n_estimators=10, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=14, min_samples_leaf=10, min_samples_split=14,
                      n_estimators=80, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=2, min_samples_split=11,
                      n_estimators=65, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=12, min_samples_leaf=2, n_estimators=50,
                      n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=6, n_estimators=75,
                      n_jobs=-1)
classifiers.append(classifier)


In [4]:
training = []
for name, var, classifier in zip(variablenam,variablelists, classifiers):
    train, val, val2 = train_val(var)
    label = (name.split('_1'))[0]
    X_train, y_train, X_val ,y_val, X_val2 ,y_val2 = X_Y(train, val, val2)
    classifier.fit(X_train, y_train)
    pr_rf = classifier.predict(X_train)
    accuracy = mean_absolute_error(y_train,pr_rf)
    training.append([name, accuracy])
    print(name, accuracy)
    predictions = classifier.predict(X_val)
    accuracy_score_metric = mean_absolute_error(y_val, predictions)
    print("Accuracy_score_metric:", accuracy_score_metric)
    predictions = classifier.predict(X_val2)
    accuracy_score_metric = mean_absolute_error(y_val2, predictions)
    print("Accuracy_score_metric:", accuracy_score_metric)
    

<ipython-input-4-266e169216b4>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


AFS_SUPERM_17 0.07432171700166174
Accuracy_score_metric: 0.08323550202084457
Accuracy_score_metric: 0.08271967709871433


<ipython-input-4-266e169216b4>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


AFS_TREINS_17 0.07336726171243602
Accuracy_score_metric: 0.08919281047447267
Accuracy_score_metric: 0.0889613583217671


<ipython-input-4-266e169216b4>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


...

logies_19 0.09007817299797877
Accuracy_score_metric: 0.10434848751177823
Accuracy_score_metric: 0.11194335347677938


<ipython-input-4-266e169216b4>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


mean_construction_year_19 10.943467825868288
Accuracy_score_metric: 11.86544973800836
Accuracy_score_metric: 11.720963641973224


<ipython-input-4-266e169216b4>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


WONING_19 2.5948088355081964
Accuracy_score_metric: 3.604250188905797
Accuracy_score_metric: 3.602884841037123


In [5]:
min_construction_year_17  = pd.read_csv("UpDistance\\min_construction_year_17_distance.csv", delimiter=',') 
max_construction_year_17  = pd.read_csv("UpDistance\\max_construction_year_17_distance.csv", delimiter=',')
min_construction_year_18  = pd.read_csv("UpDistance\\min_construction_year_18_distance.csv", delimiter=',') 
max_construction_year_18  = pd.read_csv("UpDistance\\max_construction_year_18_distance.csv", delimiter=',')
min_construction_year_19  = pd.read_csv("UpDistance\\min_construction_year_19_distance.csv", delimiter=',') 
max_construction_year_19  = pd.read_csv("UpDistance\\max_construction_year_19_distance.csv", delimiter=',')

variablelists = [min_construction_year_17,max_construction_year_17, min_construction_year_18,max_construction_year_18, 
                 min_construction_year_19,max_construction_year_19]
variablenam = ["min_construction_year_17","max_construction_year_17","min_construction_year_18","max_construction_year_18",
                "min_construction_year_19","max_construction_year_19" ]

classifiers = []
classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=9, min_samples_split=5,
                      n_estimators=95, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=7, min_samples_split=23,
                      n_estimators=55, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=9, min_samples_split=29,
                      n_estimators=55, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=6, min_samples_split=20,
                      n_estimators=90, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=3, min_samples_split=14,
                      n_estimators=70, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=5, min_samples_split=8,
                      n_estimators=65, n_jobs=-1)
classifiers.append(classifier)

In [6]:
for name, var, classifier in zip(variablenam,variablelists, classifiers):
    train, val, val2 = train_val(var)
    label = (name.split('_1'))[0]
    X_train, y_train, X_val ,y_val, X_val2 ,y_val2 = X_Y(train, val, val2)
    classifier.fit(X_train, y_train)
    pr_rf = classifier.predict(X_train)
    accuracy = mean_absolute_error(y_train,pr_rf)
    training.append([name, accuracy])
    print(name, accuracy)
    predictions = classifier.predict(X_val)
    accuracy_score_metric = mean_absolute_error(y_val, predictions)
    print("Accuracy_score_metric:", accuracy_score_metric)
    predictions = classifier.predict(X_val2)
    accuracy_score_metric = mean_absolute_error(y_val2, predictions)
    print("Accuracy_score_metric:", accuracy_score_metric)

<ipython-input-6-74afc454f5bb>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


min_construction_year_17 11.871748010986861
Accuracy_score_metric: 14.705849930170157
Accuracy_score_metric: 14.74046065179951


<ipython-input-6-74afc454f5bb>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


max_construction_year_17 9.22452679234521
Accuracy_score_metric: 11.043697349574197
Accuracy_score_metric: 11.08571785177087


<ipython-input-6-74afc454f5bb>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


...

max_construction_year_18 9.091208567363005
Accuracy_score_metric: 11.1528898764483
Accuracy_score_metric: 11.202180511529386


<ipython-input-6-74afc454f5bb>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


min_construction_year_19 11.798057115749243
Accuracy_score_metric: 14.963636740509282
Accuracy_score_metric: 14.934190620078423


<ipython-input-6-74afc454f5bb>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


max_construction_year_19 8.567560871671978
Accuracy_score_metric: 11.27185286633899
Accuracy_score_metric: 11.193221021101225


In [7]:
G_GAS_WON_17 = pd.read_csv("..\\Data\\UpDistance\\G_GAS_WON_17_distance.csv", delimiter=',') 
G_ELEK_WON_17 = pd.read_csv("..\\Data\\UpDistance\\G_ELEK_WON_17_distance.csv", delimiter=',') 
WOZWONING_17 = pd.read_csv("..\\Data\\UpDistance\\WOZWONING_17_distance.csv", delimiter=',') 
AANTAL_HH_17 = pd.read_csv("..\\Data\\UpDistance\\AANTAL_HH_17_distance.csv", delimiter=',') 
GEM_HH_GR_17 = pd.read_csv("..\\Data\\UpDistance\\GEM_HH_GR_17_distance.csv", delimiter=',') 
P_HUURWON_17 = pd.read_csv("..\\Data\\UpDistance\\P_HUURWON_17_distance.csv", delimiter=',') 
P_KOOPWON_17 = pd.read_csv("..\\Data\\UpDistance\\P_KOOPWON_17_distance.csv", delimiter=',') 

G_GAS_WON_18 = pd.read_csv("..\\Data\\UpDistance\\G_GAS_WON_18_distance.csv", delimiter=',') 
G_ELEK_WON_18 = pd.read_csv("..\\Data\\UpDistance\\G_ELEK_WON_18_distance.csv", delimiter=',') 
WOZWONING_18 = pd.read_csv("..\\Data\\UpDistance\\WOZWONING_18_distance.csv", delimiter=',') 
AANTAL_HH_18 = pd.read_csv("..\\Data\\UpDistance\\AANTAL_HH_18_distance.csv", delimiter=',') 
GEM_HH_GR_18 = pd.read_csv("..\\Data\\UpDistance\\GEM_HH_GR_18_distance.csv", delimiter=',') 
P_HUURWON_18 = pd.read_csv("..\\Data\\UpDistance\\P_HUURWON_18_distance.csv", delimiter=',') 
P_KOOPWON_18 = pd.read_csv("..\\Data\\UpDistance\\P_KOOPWON_18_distance.csv", delimiter=',') 

G_GAS_WON_19 = pd.read_csv("..\\Data\\UpDistance\\G_GAS_WON_19_distance.csv", delimiter=',') 
G_ELEK_WON_19 = pd.read_csv("..\\Data\\UpDistance\\G_ELEK_WON_19_distance.csv", delimiter=',') 
WOZWONING_19 = pd.read_csv("..\\Data\\UpDistance\\WOZWONING_19_distance.csv", delimiter=',') 
AANTAL_HH_19 = pd.read_csv("..\\Data\\UpDistance\\AANTAL_HH_19_distance.csv", delimiter=',') 
GEM_HH_GR_19 = pd.read_csv("..\\Data\\UpDistance\\GEM_HH_GR_19_distance.csv", delimiter=',') 
P_HUURWON_19 = pd.read_csv("..\\Data\\UpDistance\\P_HUURWON_19_distance.csv", delimiter=',') 
P_KOOPWON_19 = pd.read_csv("..\\Data\\UpDistance\\P_KOOPWON_19_distance.csv", delimiter=',') 

variablel_17 = [AANTAL_HH_17,GEM_HH_GR_17,P_HUURWON_17,P_KOOPWON_17,G_ELEK_WON_17, G_GAS_WON_17,WOZWONING_17]  
variablel_18 = [AANTAL_HH_18,GEM_HH_GR_18,P_HUURWON_18,P_KOOPWON_18,G_ELEK_WON_18, G_GAS_WON_18,WOZWONING_18]  
variablel_19 = [AANTAL_HH_19,GEM_HH_GR_19,P_HUURWON_19,P_KOOPWON_19,G_ELEK_WON_19, G_GAS_WON_19,WOZWONING_19] 
variablenames_17 = ["AANTAL_HH_17","GEM_HH_GR_17","P_HUURWON_17","P_KOOPWON_17","G_ELEK_WON_17", "G_GAS_WON_17",
                    "WOZWONING_17"]  
variablenames_18 = ["AANTAL_HH_18","GEM_HH_GR_18","P_HUURWON_18","P_KOOPWON_18","G_ELEK_WON_18", "G_GAS_WON_18",
                    "WOZWONING_18"]  
variablenames_19 = ["AANTAL_HH_19","GEM_HH_GR_19","P_HUURWON_19","P_KOOPWON_19","G_ELEK_WON_19", "G_GAS_WON_19",
                    "WOZWONING_19"] 

variablelists = []
for i in variablel_17:
    variablelists.append(i)
for i in variablel_18:
    variablelists.append(i)
for i in variablel_19:
    variablelists.append(i)
    
variablenames = []
for i in variablenames_17:
    variablenames.append(i)
for i in variablenames_18:
    variablenames.append(i)
for i in variablenames_19:
    variablenames.append(i)
    
classifiers = []
    
classifier = RandomForestRegressor(max_depth=14, min_samples_leaf=4, n_estimators=85,
                      n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=3, n_estimators=90,
                      n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=14, min_samples_leaf=7, min_samples_split=14,
                      n_estimators=60, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=4, min_samples_split=14,
                      n_estimators=90, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=14, min_samples_leaf=8, min_samples_split=5,
                      n_estimators=90, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=5, min_samples_split=11,
                      n_estimators=75, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=8, min_samples_split=17,
                      n_estimators=60, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=3, min_samples_split=5,
                      n_estimators=80, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=5, min_samples_split=20,
                      n_estimators=70, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=10, min_samples_split=11,
                      n_estimators=90, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=8, min_samples_split=8,
                      n_estimators=80, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=7, min_samples_split=29,
                      n_estimators=45, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=5, min_samples_split=29,
                      n_estimators=80, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=14, min_samples_leaf=10, min_samples_split=17,
                      n_estimators=85, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=4, min_samples_split=8,
                      n_estimators=40, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=6, min_samples_split=14,
                      n_estimators=80, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=5, min_samples_split=5,
                      n_estimators=90, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=14, min_samples_leaf=5, min_samples_split=11,
                      n_estimators=75, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=10, min_samples_split=20,
                      n_estimators=70, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=7, min_samples_split=8,
                      n_estimators=75, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=3, min_samples_split=8,
                      n_estimators=80, n_jobs=-1)
classifiers.append(classifier)  

In [8]:
for name, var, classifier in zip(variablenames,variablelists, classifiers):
    train, val, val2 = train_val(var)
    label = (name.split('_1'))[0]
    X_train, y_train, X_val ,y_val, X_val2 ,y_val2 = X_Y(train, val, val2)
    classifier.fit(X_train, y_train)
    pr_rf = classifier.predict(X_train)
    accuracy = mean_absolute_error(y_train,pr_rf)
    training.append([name, accuracy])
    print(name, accuracy)
    predictions = classifier.predict(X_val)
    accuracy_score_metric = mean_absolute_error(y_val, predictions)
    print("Accuracy_score_metric:", accuracy_score_metric)
    predictions = classifier.predict(X_val2)
    accuracy_score_metric = mean_absolute_error(y_val2, predictions)
    print("Accuracy_score_metric:", accuracy_score_metric)

<ipython-input-8-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


AANTAL_HH_17 1.57372109233524
Accuracy_score_metric: 1.8876524800546703
Accuracy_score_metric: 1.8071458494853032


<ipython-input-8-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


GEM_HH_GR_17 0.18519196607082972
Accuracy_score_metric: 0.2371933096403096
Accuracy_score_metric: 0.24504137966256187


<ipython-input-8-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


...

G_ELEK_WON_19 401.511727088305
Accuracy_score_metric: 485.99168548620156
Accuracy_score_metric: 482.6688307039879


<ipython-input-8-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


G_GAS_WON_19 178.04952093457985
Accuracy_score_metric: 234.4757490205459
Accuracy_score_metric: 237.64032058045083


<ipython-input-8-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


WOZWONING_19 32.770347227344175
Accuracy_score_metric: 47.91443834056677
Accuracy_score_metric: 47.5138120997281


In [9]:
P_HINK_HH_17 = pd.read_csv("..\\Data\\UpDistance\\P_HINK_HH_17_distance.csv", delimiter=',') 
P_LINK_HH_17 = pd.read_csv("..\\Data\\UpDistance\\P_LINK_HH_17_distance.csv", delimiter=',') 
WON_NBEW_17 = pd.read_csv("..\\Data\\UpDistance\\WON_NBEW_17_distance.csv", delimiter=',')  

P_HINK_HH_18 = pd.read_csv("..\\Data\\UpDistance\\P_HINK_HH_18_distance.csv", delimiter=',') 
P_LINK_HH_18 = pd.read_csv("..\\Data\\UpDistance\\P_LINK_HH_18_distance.csv", delimiter=',') 
WON_NBEW_18 = pd.read_csv("..\\Data\\UpDistance\\WON_NBEW_18_distance.csv", delimiter=',') 

P_HINK_HH_19 = pd.read_csv("..\\Data\\UpDistance\\P_HINK_HH_19_distance.csv", delimiter=',') 
P_LINK_HH_19 = pd.read_csv("..\\Data\\UpDistance\\P_LINK_HH_19_distance.csv", delimiter=',') 
WON_NBEW_19 = pd.read_csv("..\\Data\\UpDistance\\WON_NBEW_19_distance.csv", delimiter=',') 

variablel_17 = [P_HINK_HH_17 ,P_LINK_HH_17 ,WON_NBEW_17]
variablel_18 = [P_HINK_HH_18 ,P_LINK_HH_18 ,WON_NBEW_18]               
variablel_19 = [P_HINK_HH_19 ,P_LINK_HH_19 ,WON_NBEW_19]
variablenames_17 = ["P_HINK_HH_17","P_LINK_HH_17","WON_NBEW_17"]
variablenames_18 = ["P_HINK_HH_18","P_LINK_HH_18","WON_NBEW_18"]
variablenames_19 = ["P_HINK_HH_19","P_LINK_HH_19","WON_NBEW_19"]

variablelists = []
for i in variablel_17:
    variablelists.append(i)
for i in variablel_18:
    variablelists.append(i)
for i in variablel_19:
    variablelists.append(i)
    
variablenames = []
for i in variablenames_17:
    variablenames.append(i)
for i in variablenames_18:
    variablenames.append(i)
for i in variablenames_19:
    variablenames.append(i)

classifiers = []
classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=3, n_estimators=50,
                      n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_split=8, n_estimators=50,
                      n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=4, min_samples_split=8,
                      n_estimators=60, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=3, min_samples_split=11,
                      n_estimators=90, n_jobs=-1)
classifiers.append(classifier)


classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=3, min_samples_split=26,
                      n_estimators=25, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=16, min_samples_leaf=5, min_samples_split=11,
                      n_estimators=40, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=8, min_samples_split=14,
                      n_estimators=35, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=5, min_samples_split=8,
                      n_estimators=70, n_jobs=-1)
classifiers.append(classifier)

classifier = RandomForestRegressor(max_depth=18, min_samples_leaf=4, min_samples_split=23,
                      n_estimators=80, n_jobs=-1)
classifiers.append(classifier)

In [10]:
for name, var, classifier in zip(variablenames,variablelists, classifiers):
    train, val, val2 = train_val(var)
    label = (name.split('_1'))[0]
    X_train, y_train, X_val ,y_val, X_val2 ,y_val2 = X_Y(train, val, val2)
    classifier.fit(X_train, y_train)
    pr_rf = classifier.predict(X_train)
    accuracy = mean_absolute_error(y_train,pr_rf)
    training.append([name, accuracy])
    print(name, accuracy)
    predictions = classifier.predict(X_val)
    accuracy_score_metric = mean_absolute_error(y_val, predictions)
    print("Accuracy_score_metric:", accuracy_score_metric)
    predictions = classifier.predict(X_val2)
    accuracy_score_metric = mean_absolute_error(y_val2, predictions)
    print("Accuracy_score_metric:", accuracy_score_metric)

<ipython-input-10-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


P_HINK_HH_17 0.4347301046608438
Accuracy_score_metric: 0.6806218199869581
Accuracy_score_metric: 0.6851961113843124


<ipython-input-10-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


P_LINK_HH_17 0.5918597344280548
Accuracy_score_metric: 1.0067724580501842
Accuracy_score_metric: 1.0418520924841959


<ipython-input-10-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


...

P_HINK_HH_19 0.5853776365400805
Accuracy_score_metric: 0.7225691503212649
Accuracy_score_metric: 0.7119360913038374


<ipython-input-10-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


P_LINK_HH_19 0.7284862444148424
Accuracy_score_metric: 1.0128421637038425
Accuracy_score_metric: 1.0110149018333847


<ipython-input-10-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


WON_NBEW_19 0.28243002634303455
Accuracy_score_metric: 0.33281421358864993
Accuracy_score_metric: 0.42909288429452447


In [15]:
UITKMINAOW_17 = pd.read_csv("..\\Data\\UpDistance\\UITKMINAOW_17_distance.csv", delimiter=',')

UITKMINAOW_18 = pd.read_csv("..\\Data\\UpDistance\\UITKMINAOW_18_distance.csv", delimiter=',') 

UITKMINAOW_19 = pd.read_csv("..\\Data\\UpDistance\\UITKMINAOW_19_distance.csv", delimiter=',') 

variablenames_17 = ["UITKMINAOW_17"]
variablenames_18 = ["UITKMINAOW_18"]
variablenames_19 = ["UITKMINAOW_19"]

variablelists = [UITKMINAOW_17 ,UITKMINAOW_18,UITKMINAOW_19]
    
variablenames = []
for i in variablenames_17:
    variablenames.append(i)
for i in variablenames_18:
    variablenames.append(i)
for i in variablenames_19:
    variablenames.append(i)
    
classifiers = []
classifier = RandomForestRegressor(max_depth = 16, min_samples_leaf= 10, min_samples_split = 23, n_estimators = 90)
classifiers.append(classifier)
    
classifier = RandomForestRegressor(max_depth = 18, min_samples_leaf= 8, min_samples_split = 23, n_estimators = 45)
classifiers.append(classifier)
                            
classifier = RandomForestRegressor(max_depth = 18, min_samples_leaf=8, min_samples_split = 8, n_estimators = 95)
classifiers.append(classifier)

In [16]:
for name, var, classifier in zip(variablenames,variablelists, classifiers):
    train, val, val2 = train_val(var)
    label = (name.split('_1'))[0]
    X_train, y_train, X_val ,y_val, X_val2 ,y_val2 = X_Y(train, val, val2)
    classifier.fit(X_train, y_train)
    pr_rf = classifier.predict(X_train)
    accuracy = mean_absolute_error(y_train,pr_rf)
    training.append([name, accuracy])
    print(name, accuracy)
    predictions = classifier.predict(X_val)
    accuracy_score_metric = mean_absolute_error(y_val, predictions)
    print("Accuracy_score_metric:", accuracy_score_metric)
    predictions = classifier.predict(X_val2)
    accuracy_score_metric = mean_absolute_error(y_val2, predictions)
    print("Accuracy_score_metric:", accuracy_score_metric)

<ipython-input-16-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


UITKMINAOW_17 4.367571451443868
Accuracy_score_metric: 5.25412398796004
Accuracy_score_metric: 5.277313722222689


<ipython-input-16-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


UITKMINAOW_18 4.26651710901608
Accuracy_score_metric: 5.264950856025313
Accuracy_score_metric: 5.339042745961849


<ipython-input-16-77c0345a30c2>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


UITKMINAOW_19 4.043621850049587
Accuracy_score_metric: 5.144367056838285
Accuracy_score_metric: 5.176323649673048


In [17]:
print(training)

[['AFS_SUPERM_17', 0.07432171700166174], ['AFS_TREINS_17', 0.07336726171243602], ['AFS_ONDBAS_17', 0.07343219021839305], ['AFS_ONDVRT_17', 0.0729556399522285], ['AFS_HAPRAK_17', 0.07275335268154379], ['AFS_ZIEK_E_17', 0.07808036712574078], ['AFS_HORECA_17', 0.07580231009898908], ['woon_17', 8.927870103241265], ['winkel_17', 0.404778850978181], ['gezondheid_17', 0.2603737351802509], ['kantoor_17', 0.24972509515484925], ['bijeenkomst_17', 0.19190347801985902], ['onderwijs_17', 0.07169097791046147], ['sport_17', 0.025093960673360644], ['industrie_17', 0.3049486977857213], ['logies_17', 0.0881081304282323], ['mean_construction_year_17', 10.572616275713825], ['WONING_17', 2.8222094141905294], ['woon_18', 8.997178635235192], ['winkel_18', 0.41180040834148784], ['gezondheid_18', 0.2677223877598375], ['kantoor_18', 0.23847049779153898], ['bijeenkomst_18', 0.19387337786188757], ['onderwijs_18', 0.06704212716652506], ['sport_18', 0.025601513129442078], ['industrie_18', 0.30855617615228315], ['lo